# Stage 6

## Running the experiments

In the previous stage, our team selected `Sammon's error` function and `Genetic algorithm` and `Simulated annealing` for the experiments, so we wil use them here.

### 1. Loading and splitting the data

In [52]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [53]:
# Loading the data
df = pd.read_csv('combined-metrics.csv')

In [54]:
X = df.drop('Name', axis=1).values
y = df['Name'].values

In [55]:
print(X.shape)

(101, 38)


In [56]:
X

array([[5.42625369e+00, 3.77802360e+01, 2.97935103e+01, ...,
        1.81400000e+03, 1.58500000e+04, 0.00000000e+00],
       [2.02732240e+00, 9.52868852e+00, 6.37568306e+00, ...,
        9.22000000e+02, 2.64500000e+03, 0.00000000e+00],
       [1.67987805e+00, 1.17048780e+01, 8.17073171e+00, ...,
        2.79100000e+03, 1.14790000e+04, 0.00000000e+00],
       ...,
       [4.29198473e+00, 2.41488550e+01, 1.87232824e+01, ...,
        3.82000000e+02, 6.43600000e+03, 0.00000000e+00],
       [9.15789474e+00, 3.77017544e+01, 2.96842105e+01, ...,
        1.78000000e+02, 1.38800000e+03, 0.00000000e+00],
       [2.24528302e+00, 1.04371069e+01, 7.09119497e+00, ...,
        8.01000000e+02, 2.98200000e+03, 0.00000000e+00]])

In [57]:
y

array(['ART', 'BentoML', 'Bokeh', 'Camel', 'CatBoost', 'Causal ML',
       'Chainer', 'Computer Vision', 'D2L', 'Darts', 'Dash', 'DeepChem',
       'DeepMind Control', 'DeepPavlov', 'Detectron2', 'DIGITS',
       'DragGan', 'EasyOCR', 'ELI5', 'EvalAI', 'facenet', 'FaceSwap',
       'Fairseq', 'FastAI', 'FeatureTools', 'FiftyOne', 'gensim',
       'Giskard', 'Gluonts', 'Google Flax', 'Google JAX', 'GPT-Engineer',
       'GPTDiscord', 'Gradio', 'Gymnasium', 'Horovod', 'ImageAI',
       'imbalanced-learn', 'InsightFace', 'Kaolin', 'Kedro', 'Keras',
       'Kserve', 'Lightning', 'Ludwig', 'Mage-ai', 'Mars', 'Matplotlib',
       'metatransformer', 'Mindsdb', 'MLflow', 'Mycroft',
       'Neural Prophet', 'NNI', 'Numpy', 'ONNX', 'Open-Assistant',
       'OpenAI Baselines', 'OpenAI Python API library', 'OpenVINO',
       'Optuna', 'Paddle', 'Pandas', 'Pocker', 'Pybrain', 'PyCaret',
       'pycm', 'PyMC', 'Pyro', 'PyTensor', 'PyTorch',
       'Pytorch image models', 'qlib', 'Rasa', 'Ray', 'Reco

In [58]:
# Splitting the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)

(80, 38)
(21, 38)


### 2. Determining the minimal subset of metrics on the training data

Sammon's error function that we will use as an objective

In [59]:
import numpy as np

# Sammon's error function
def sammons_error(X, Y):
    """
    X: Original high-dimensional data
    Y: Low-dimensional representation
    """
    # Calculate pairwise distances in X and Y
    dist_orig = np.sqrt(np.sum((X[:, np.newaxis, :] - X[np.newaxis, :, :]) ** 2, axis=-1))
    dist_lowd = np.sqrt(np.sum((Y[:, np.newaxis, :] - Y[np.newaxis, :, :]) ** 2, axis=-1))

    # Avoid division by zero
    epsilon = 1e-12

    numerator = np.sum(((dist_orig - dist_lowd) * 2) / (dist_orig + epsilon))
    denominator = np.sum(dist_orig)

    error = numerator / denominator

    return error

#### Genetic algorithm

In [60]:
def fitness_genetic(individual, X):
    # Create a subset of features based on the individual
    subset_X = X[:, np.array(individual).astype(bool)]

    # Calculate the fitness using the `sammons_error` function
    fitness = -sammons_error(X, subset_X)
    return fitness,

In [ ]:
from deap import creator, base, tools, algorithms
import random
from sklearn.decomposition import PCA

# Set up the genetic algorithm
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


toolbox = base.Toolbox()


# Functions to limit the number of features in the individual to 25
def create_individual():
    individual = [random.randint(0, 1) for _ in range(X.shape[1])]
    while sum(individual) > 25:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(individual) - 25, replace=False)
        for idx in indices:
            individual[idx] = 0
    return creator.Individual(individual)


def mate(ind1, ind2):
    child1, child2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
    tools.cxTwoPoint(child1, child2)

    # Limit the number of features in the child to 25
    while sum(child1) > 25:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(child1) - 25, replace=False)
        for idx in indices:
            child1[idx] = 0
    while sum(child2) > 25:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(child2) - 25, replace=False)
        for idx in indices:
            child2[idx] = 0
    return child1, child2


def mutate(individual):
    child = toolbox.clone(individual)
    tools.mutFlipBit(child, indpb=0.05)
    while sum(child) > 25:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(child) - 25, replace=False)
        for idx in indices:
            child[idx] = 0
    return child,


toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_genetic, X=X_train)
toolbox.register("mate", mate)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

In [62]:
import numpy as np

# Run the genetic algorithm
population = toolbox.population(n=100)
hof = tools.HallOfFame(100)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

population, log = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)

gen	nevals	avg         	std        	min         	max         
0  	100   	-1.93929e-06	1.39794e-06	-5.01034e-06	-7.44105e-08
1  	66    	-7.52009e-07	5.98858e-07	-2.45842e-06	-5.11421e-08


2  	63    	-3.21586e-07	3.01651e-07	-2.04449e-06	-5.09807e-08
3  	65    	-1.93544e-07	2.01597e-07	-1.8389e-06 	-1.56846e-08
4  	58    	-1.33633e-07	2.55222e-07	-1.86428e-06	-1.56846e-08
5  	65    	-6.81563e-08	6.17763e-08	-3.2524e-07 	-4.67344e-10
6  	61    	-6.14201e-08	1.53245e-07	-1.20946e-06	-4.61628e-10
7  	55    	-7.48719e-08	2.66823e-07	-1.79787e-06	-4.61628e-10
8  	55    	-5.87705e-08	2.48812e-07	-1.53183e-06	-4.61628e-10
9  	59    	-5.00508e-08	2.3234e-07 	-1.53164e-06	-3.08197e-10
10 	67    	-9.44696e-09	4.29224e-08	-3.24233e-07	-4.57732e-10
11 	53    	-7.5635e-08 	4.18026e-07	-3.78036e-06	-1.68474e-10
12 	72    	-4.85894e-08	2.36694e-07	-1.5596e-06 	-1.9908e-10 
13 	60    	-5.39993e-08	2.61567e-07	-1.53182e-06	-1.9908e-10 
14 	54    	-4.23305e-08	2.21427e-07	-1.79753e-06	-1.98518e-10
15 	61    	-6.81634e-08	2.89012e-07	-1.52602e-06	-3.96452e-11
16 	63    	-3.38642e-08	1.59259e-07	-1.52579e-06	-3.96452e-11
17 	63    	-7.55579e-08	2.66544e-07	-1.52633e-06	-3.96452e-11
18 	57  

In [63]:
# Get the best individual (subset of metrics)
best_individual = hof.items[0]
best_subset_genetic = [df.columns[i+1] for i, bit in enumerate(best_individual) if bit]

print("Best subset of metrics:", best_subset_genetic)
print("Best fitness:", fitness_genetic(best_individual, X_train))
print("Length of best subset:", len(best_subset_genetic))

Best subset of metrics: ['h2', 'N1', 'N2', 'length', 'volume', 'effort', 'time', 'CCL', 'CCO', 'CEE', 'CI', 'LDC', 'LLDC', 'TLLOC', 'TLOC', 'TNCL', 'TNFI', 'TNM', 'TNOS', 'TNPKG', 'TCLOC', 'TNDI', 'WarningCritical', 'WarningMajor', 'WarningMinor']
Best fitness: (-2.013710327311463e-12,)
Length of best subset: 25


#### Simulated annealing

In [64]:
import math

def fitness_annealing(X, subset_X):
    return -sammons_error(X, subset_X)

def accept_probability(curr_score, best_score, temperature):
    if curr_score > best_score:
        return 1.0
    else:
        return math.exp((curr_score - best_score) / temperature)
    
def generate_random_subset(X):
    # Generate new random subset
    subset = np.random.randint(2, size=(38,)) # 0 or 1 masks

    # Limit the number of features in the subset to 25
    while sum(subset) > 25:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(subset) - 25, replace=False)
        for idx in indices:
            subset[idx] = 0

    return subset

def annealing(X):
    # Simulated annealing parameters
    t = 1000 # Initial temperature
    t_min = 1e-3 # Minimum temperature
    cooling_factor = 0.99 # Temperature damping factor
    max_iterations = 1000 # Maximum number of iterations
    best_subset = None
    best_score = float("-inf")
    
    for _ in range(max_iterations):

        subset = generate_random_subset(X) # Generate random subset    

        score = fitness_annealing(X, X[:,subset==1]) # Evaluate subset

        # Update best, higher is better
        if score > best_score or random.random() < accept_probability(score, best_score, t):
            best_subset = subset
            best_score = score

        # Cool temperature
        t = t * cooling_factor

        # Check if cooled enough
        if t < t_min:
            break

    return best_subset, best_score

best_subset, best_score = annealing(X_train)

print("Best subset:", best_subset)
print("Best sammon's error result:", -best_score)

Best subset: [1 0 1 0 1 0 0 1 1 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 1
 0]
Best sammon's error result: 4.197337155943921e-06


In [65]:
best_subset_annealing = [df.columns[i+1] for i, bit in enumerate(best_subset) if bit]
print(best_subset_annealing)
print(len(best_subset_annealing))

['h1', 'N1', 'vocabulary', 'difficulty', 'effort', 'bugs', 'MI', 'CC', 'CCO', 'CI', 'CLC', 'CR', 'TNCL', 'TNM', 'TNPKG', 'TCLOC', 'WarningBlocker', 'WarningMinor']
18


In [66]:
print("Genetic algorithm result: ", best_subset_annealing)
print("Simulated annealing result:", best_subset_genetic)



X_genetic = X_train[:, np.array(best_individual).astype(bool)]
X_annealing = X_train[:, np.array(best_subset).astype(bool)]

X_genetic_test = X_test[:, np.array(best_individual).astype(bool)]
X_annealing_test = X_test[:, np.array(best_subset).astype(bool)]

print("Genetic training score:", sammons_error(X_train, X_genetic))
print("Annealing training score:", sammons_error(X_train, X_annealing))

print("Genetic validation score:", sammons_error(X_test, X_genetic_test))
print("Annealing validation score:", sammons_error(X_test, X_annealing_test))

Genetic algorithm result:  ['h1', 'N1', 'vocabulary', 'difficulty', 'effort', 'bugs', 'MI', 'CC', 'CCO', 'CI', 'CLC', 'CR', 'TNCL', 'TNM', 'TNPKG', 'TCLOC', 'WarningBlocker', 'WarningMinor']
Simulated annealing result: ['h2', 'N1', 'N2', 'length', 'volume', 'effort', 'time', 'CCL', 'CCO', 'CEE', 'CI', 'LDC', 'LLDC', 'TLLOC', 'TLOC', 'TNCL', 'TNFI', 'TNM', 'TNOS', 'TNPKG', 'TCLOC', 'TNDI', 'WarningCritical', 'WarningMajor', 'WarningMinor']
Genetic training score: 2.013710327311463e-12
Annealing training score: 4.197337155943921e-06
Genetic validation score: 2.9773385857308484e-12
Annealing validation score: 3.2220047850834426e-06
